In [ ]:
def main():
    pass

if __name__ == '__main__':
    main()

import aiohttp
import asyncio
import io
import requests
import csv
import os
import json
import sys
import pandas as pd
import time
import subprocess
import calendar
from typing import List, Dict
from pandas import json_normalize
from datetime import date
from datetime import timedelta
import datetime
import pytz
import win32com.client
import plotly
import dateutil
from dateutil import parser as dtparse
from functools import partial

import nest_asyncio
nest_asyncio.apply()

from plotly import offline
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash

from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import voila 

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"   # suggested to add this so html rendering matches notebook
#pio.renderers.default = "notebook_connected

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'


UNIT = "Minute"
INTERVAL = 1
BARS_BACK = 10
    


class Client:
    def __init__(self, client_id: str, client_secret: str, refresh_token: str, expiration_buffer: int = 30) -> None:
        self.client_id = client_id
        self.client_secret = client_secret
        self.refresh_token = refresh_token
        self.access_token = ""
        self.expiration_buffer = expiration_buffer
        self.expires_in = 0
        self.time_of_last_refresh = 0
        self.auth_url = "https://signin.tradestation.com/oauth/token"

    async def get_access_token(self, force: bool = False) -> str:
        seconds_since_last_token = time.time() - self.time_of_last_refresh

        if force or seconds_since_last_token > self.expires_in - self.expiration_buffer:
            payload = {
                "grant_type": "refresh_token",
                "client_id": self.client_id,
                "client_secret": self.client_secret,
                "refresh_token": self.refresh_token,
            }


            async with aiohttp.ClientSession() as session:
                  async with session.post(self.auth_url, data=payload) as response:
                    json_data = await response.json()
                    self.time_of_last_refresh = time.time()
                    self.expires_in = int(json_data["expires_in"])
                    self.access_token = json_data["access_token"]

        return self.access_token


app = Dash(__name__)
bars = []

#################################################

app.layout = html.Div([
    dcc.Graph(id='live-chart'),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # in milliseconds
        n_intervals=0
    )
])

@app.callback(
    Output('live-chart', 'figure'),
    Input('interval-component', 'n_intervals')
)


def update_chart(n):
    if not bars:
        return go.Figure()
    try:
        df = pd.DataFrame(bars)

        Central = pytz.timezone('US/Central')
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
        df['TimeStamp'] = df['TimeStamp'].replace("T", " ").replace("Z", "")
        df['TimeStamp'] = df['TimeStamp'].dt.tz_convert(Central)
        
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'].dt.strftime('%Y-%m-%d %H:%M:%S'))
        df = df.sort_values('TimeStamp', ascending=True)
        df['Hour'] = (df['TimeStamp'].dt.strftime('%I:%M'))
        
        df2 = df[['Open', 'High', 'Low', 'Close', 'TotalVolume']].copy()
        df2 = df2.astype(float)
        df2['Hour'] = df['Hour']
        #df2 = df2.sort_values('Hour', ascending=True)
        
        df2 = df2.reset_index(drop=True)

        df2 = df2.tail(BARS_BACK)
        
        df2.columns =['Open', 'High', 'Low', 'Close', 'TotalVolume', 'Hour']
        df2 = df2.iloc[: ,:].head(10)
        df2 = df2.sort_values('Hour', ascending=True)
        #print(df2)

     
        #sym_str = str(symbol)

             
        fig = make_subplots(rows=2, cols=1, 
                            shared_xaxes=True, 
                            vertical_spacing=0.10, 
                            subplot_titles=('Price', 'Volume'), 
                            row_width=[0.3, 0.7])
        
        df2['MA10'] = df2['Close'].rolling(window=10, min_periods=0).mean() 
        
                
               
        fig.add_trace(go.Candlestick(x=df2.index, 
                                    open=df2['Open'], 
                                    high=df2['High'],
                                    low=df2['Low'], 
                                    close=df2['Close'], 
                                    increasing_line_color = 'rgb(28,28,107)',
                                    decreasing_line_color = 'rgb(68,68,112)'), 
                    row=1, col=1)
                   
        
        fig.add_trace(go.Scatter(x=df2.index, 
                                y=df2["MA10"], 
                                marker_color='aqua',
                                name="MA10"), 
                        row=1, col=1)

        fig.add_trace(go.Bar(x=df2['Hour'], 
                                y=df2['TotalVolume'], 
                                marker_color='red', 
                                showlegend=False), 
                        row=2, col=1)
               
        fig.update_layout(
                            title=f'Historical Price Chart',
                            yaxis_title='Price ($/share)',
                            yaxis2_title='Volume',
                            xaxis_tickfont_size=12,
                            autosize=False,
                            width=800,
                            height=500,
                            margin=dict(l=50, r=50, b=100, t=100, pad=4),
                            plot_bgcolor= 'rgba(0,0,0,0)',
                            paper_bgcolor= 'rgba(0,0,0,0)'   
                            )
    
                            
        
        fig.update_yaxes(type='linear', row=2, col=1)
        
        fig.update_yaxes(title_text="Price", row=1, col=1)
              
        fig.update_yaxes(title_text="Volume", row=2, col=1)
            
        fig.update(layout_xaxis_rangeslider_visible=False)
        
        return fig 

    except Exception as e:
        print(f"Error fetching data: {e}")
        #await asyncio.sleep(1)  


async def fetch_and_process_data(symbol: str, client: Client, unit: str, interval: int, bars_back: int):
    global bars
    while True:
        url = f"https://api.tradestation.com/v3/marketdata/stream/barcharts/{symbol}"
        params = {
            "interval": interval,
            "unit": unit,
            "barsback": bars_back,
        }
            
        access_token = await client.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}

        try:
            async with aiohttp.ClientSession() as session:
                async with session.get(url, params=params, headers=headers) as response:
                    async for line in response.content:
                        if line:
                            bar = json.loads(line.decode())
                            if "Heartbeat" not in bar:
                                if len(bars) >= bars_back:
                                    bars.pop(0)
                                    
                                if len(bars) == 0 or bars[-1]["Epoch"] != bar["Epoch"]:
                                    bars.append(bar)
                                else:
                                    bars[-1] = bar
                                    
                                df = pd.DataFrame(bars)  
                                
        
        except Exception as e:
            print(f"Error fetching data: {e}")
            await asyncio.sleep(1)

# def update_chart(n):
#     if not bars:
#         return go.Figure()
#     try:
#         df = pd.DataFrame(bars)

#         Central = pytz.timezone('US/Central')
#         df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
#         df['TimeStamp'] = df['TimeStamp'].replace("T", " ").replace("Z", "")
#         df['TimeStamp'] = df['TimeStamp'].dt.tz_convert(Central)
        
#         df['TimeStamp'] = pd.to_datetime(df['TimeStamp'].dt.strftime('%Y-%m-%d %H:%M:%S'))
#         df['Hour'] = (df['TimeStamp'].dt.strftime('%I:%M'))
        
#         df2 = df[['Open', 'High', 'Low', 'Close', 'TotalVolume']].copy()
#         df2 = df2.astype(float)
#         df2['Hour'] = df['Hour']
#         df2 = df2.sort_values('Hour', ascending=True)

        
#         df2.columns =['Open', 'High', 'Low', 'Close', 'TotalVolume', 'Hour']
#         df2 = df2.iloc[: ,:].head(10)
#         df2 = df2.sort_values('Hour', ascending=True)

     
#         sym_str = str(symbol)

        
#         fig = make_subplots(rows=2, cols=1, 
#                             shared_xaxes=True, 
#                             vertical_spacing=0.10, 
#                             subplot_titles=('Price', 'Volume'), 
#                             row_width=[0.3, 0.7])
        
#         df2['MA10'] = df2['Close'].rolling(window=10, min_periods=0).mean() 
        
                
               
#         fig.add_trace(go.Candlestick(x=df2.index, 
#                                     open=df2['Open'], 
#                                     high=df2['High'],
#                                     low=df2['Low'], 
#                                     close=df2['Close'], 
#                                     increasing_line_color = 'rgb(28,28,107)',
#                                     decreasing_line_color = 'rgb(68,68,112)'), 
#                     row=1, col=1)
                   
        
#         fig.add_trace(go.Scatter(x=df2.index, 
#                                 y=df2["MA10"], 
#                                 marker_color='aqua',
#                                 name="MA10"), 
#                         row=1, col=1)

#         fig.add_trace(go.Bar(x=df2['Hour'], 
#                                 y=df2['TotalVolume'], 
#                                 marker_color='red', 
#                                 showlegend=False), 
#                         row=2, col=1)
               
#         fig.update_layout(
#                             title=f'{sym_str} Historical Price Chart',
#                             yaxis_title='Price ($/share)',
#                             yaxis2_title='Volume',
#                             xaxis_tickfont_size=12,
#                             autosize=False,
#                             width=800,
#                             height=500,
#                             margin=dict(l=50, r=50, b=100, t=100, pad=4),
#                             plot_bgcolor= 'rgba(0,0,0,0)',
#                             paper_bgcolor= 'rgba(0,0,0,0)'   
#                             )
    
                            
        
#         fig.update_yaxes(type='linear', row=2, col=1)
        
#         fig.update_yaxes(title_text="Price", row=1, col=1)
              
#         fig.update_yaxes(title_text="Volume", row=2, col=1)
            
#         fig.update(layout_xaxis_rangeslider_visible=False)
        
#         return fig 

    
        # timezone = pytz.timezone("US/Central")  
        # now = datetime.datetime.now()
        # now = timezone.localize(now)
        # ThisTime = datetime.datetime.strftime(now, "%Y-%m-%d %H:%M:%S %p")
        
               
        # Closing = datetime.datetime.now().replace(hour=15, minute=00, second=0, microsecond=0, )
        # Closing = timezone.localize(Closing)
        # Closing = datetime.datetime.strftime(Closing, "%Y-%m-%d %H:%M:%S %p")
              
        # TimeStart = ThisTime
        # TimeEnd = Closing
        # TimeEnd = datetime.datetime.strptime(Closing, "%Y-%m-%d %H:%M:%S %p")
        # TimeStart = datetime.datetime.strptime(ThisTime, "%Y-%m-%d %H:%M:%S %p")
               
        # if TimeStart > TimeEnd:
        #     return True
        # else:
        #     return False



       
        #fig.write_html("c:\github\git\AAPL.html")
        #plotly.io.write_html(fig, 'c:\AAPL.html')
        #fig.write_html


        #plotly.offline.plot(fig, filename='C:\github\git\AAPL.html')
        
        #fig.write_html(fig, filename=APPL.html')
        #plotly.offline.plot(fig, filename='https://Pancake205.github.io')

        plotly.offline.plot(f2, filename='c:\\users\\BeckyMcFarland\\AppData\\Local\\Programs\\Python\chart.html')
        #fig.show(renderer="colab")



async def main():
 
    cols = ['SYMBOL']
    df = pd.read_csv('c:\\Users\\BeckyMcFarland\\AppData\\Local\\Programs\\Python\\Python312\\SCPSymbols.csv', names=cols)
  
    
    client = Client(
                client_id="9yqHGFAuqBulr0Fdw6z0bp6Cj4jD1Zzb",
                client_secret="Pnduh5O1PORKocWafm6P5ZR1-sw6DS4PDSPwkgZYI4IBxYGBREoMlto7r_FVBVkq",
                refresh_token="Nr8a_hfH_q2xzp62o8Ux1UYwFCsYra6DyHlaxWEA-9o3e"
    )

    async def fetch_data_loop(df, client):
        tasks = []
        for _, row in df.iterrows():
            symbol = row['SYMBOL']
            task = asyncio.create_task(fetch_and_process_data(symbol, client, UNIT, INTERVAL, BARS_BACK))
            tasks.append(task)
        return await asyncio.gather(*tasks)            
            
# Start the Dash server in the main thread
    fetch_task = asyncio.create_task(fetch_data_loop(df, client))
    app.run_server(debug=False, port=8080, use_reloader=False)
    await fetch_task 
    # Run both tasks
    
    
    async def run_dash_server():
        app.run_server(debug=False, port=8080)
#http://127.0.0.1:8080/

   
if __name__ == "__main__":
    asyncio.run(main())
   
    


